# Resouce estimation

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from custom_decompositions import *

## Original ansatze

In [2]:
from ansatze_custom_heavyopt_decomposed import *

ansatz_map = {
    18: [ansatz_18O, 5],
    20: [ansatz_20O, 14],
    22: [ansatz_22O, 19],
    24: [ansatz_24O, 14],
    26: [ansatz_26O, 5]
}

In [3]:
dev = qml.device("default.qubit", wires=12)

In [4]:
for isotope in ansatz_map.keys():
    ansatz, num_params = ansatz_map[isotope]

    @qml.qnode(dev, expansion_strategy="gradient")
    def cost(params):
        ansatz(params)
        return qml.expval(qml.PauliZ(0))   
    
    print(f"Isotope {isotope}") 
    pprint(qml.specs(cost, expansion_strategy="device")(np.zeros(num_params)))
    print()

Isotope 18
{'depth': 15,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'gradient',
 'gate_sizes': defaultdict(<class 'int'>, {1: 11, 2: 23}),
 'gate_types': defaultdict(<class 'int'>, {'PauliX': 2, 'RY': 9, 'CNOT': 23}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'auto',
 'num_device_wires': 12,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 34,
 'num_trainable_params': 9,
 'num_used_wires': 12,
 'resources': Resources(num_wires=12,
                        num_gates=34,
                        gate_types=defaultdict(<class 'int'>,
                                               {'CNOT': 23,
                                                'PauliX': 2,
                                                'RY': 9}),
                        gate_sizes=defaultdict(<class 'int'>, {1: 11, 2: 23}),
                        depth=15,
                        shots=Shots(total_shots=None, shot_vector=()))}

Isot

## Optimized w/Qiskit transpiler

In [5]:
isotope = 18

ansatz, num_params = ansatz_map[isotope]

@qml.qfunc_transform
def expand_rot_and_remove_zeros(tape):
    for op in tape:
        if op.name == "Rot":
            if not np.isclose(op.data[0], 0.0):
                qml.RZ(op.data[0], wires=op.wires)
            if not np.isclose(op.data[1], 0.0):
                qml.RY(op.data[1], wires=op.wires)
            if not np.isclose(op.data[2], 0.0):
                qml.RZ(op.data[2], wires=op.wires)
        else:
            qml.apply(op)

@qml.qfunc_transform
def expand_paulix(tape):
    for op in tape:
        if op.name == "PauliX":
            qml.RZ(np.pi/2, wires=op.wires)
            qml.RY(np.pi, wires=op.wires)
            qml.RZ(-np.pi/2, wires=op.wires)
        else:
            qml.apply(op)
            
pipeline = [
    expand_paulix,
    qml.transforms.cancel_inverses,
    qml.transforms.single_qubit_fusion(),
    expand_rot_and_remove_zeros,
    qml.transforms.merge_rotations(),
    qml.transforms.cancel_inverses,
]

@qml.qnode(dev)
@qml.compile(pipeline=pipeline)
def cost(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0))   

params = np.random.uniform(size=num_params)

pprint(qml.specs(cost, expansion_strategy="device")(params))

{'depth': 15,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'gradient',
 'gate_sizes': defaultdict(<class 'int'>, {1: 13, 2: 23}),
 'gate_types': defaultdict(<class 'int'>, {'RZ': 2, 'RY': 11, 'CNOT': 23}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'auto',
 'num_device_wires': 12,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 36,
 'num_trainable_params': 9,
 'num_used_wires': 12,
 'resources': Resources(num_wires=12,
                        num_gates=36,
                        gate_types=defaultdict(<class 'int'>,
                                               {'CNOT': 23,
                                                'RY': 11,
                                                'RZ': 2}),
                        gate_sizes=defaultdict(<class 'int'>, {1: 13, 2: 23}),
                        depth=15,
                        shots=Shots(total_shots=None, shot_vector=()))}


In [6]:
print(qml.draw(cost, expansion_strategy="device")(params))

 0: ──RZ(3.14)───RY(3.14)───────────╭X──────────────────────────────────────────────────────────┤
 1: ──RZ(3.14)───RY(3.14)───────────│──╭X───────────────────────────────────────────────────────┤
 2: ──────────────────────╭X────────╰●─│────────────╭X──────────────────────────────────────────┤
 3: ──RY(0.53)──╭●────────╰●────────╭●─╰●───────────│──╭X───────────────────────────────────────┤
 4: ────────────│───────────────────│─────╭X────────╰●─│────────────╭X──────────────────────────┤
 5: ──RY(-0.27)─╰X─────────RY(0.27)─╰X─╭●─╰●────────╭●─╰●───────────│──╭X───────────────────────┤
 6: ───────────────────────────────────│────────────│─────╭X────────╰●─│────────────╭X──────────┤
 7: ──RY(-0.11)────────────────────────╰X──RY(0.11)─╰X─╭●─╰●────────╭●─╰●───────────│──╭X───────┤
 8: ───────────────────────────────────────────────────│────────────│─────╭X────────╰●─│──╭X────┤
 9: ──RY(-0.04)────────────────────────────────────────╰X──RY(0.04)─╰X─╭●─╰●────────╭●─╰●─│──╭X─┤
10: ────────────────

In [7]:
from qiskit import QuantumCircuit
from qiskit import transpile

# Get the tape and run it through the transpiler; use the optimal
# params but convert to non-diff version
original_tape = cost.qtape
original_qasm = original_tape.to_openqasm()

qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

In [8]:
@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    return qml.expval(qml.PauliZ(0))

In [9]:
qml.specs(tapered_circuit_transpiled)()

/home/olivia/.conda/envs/ionizer/lib/python3.11/site-packages/pennylane_qiskit/converter.py:227: UserWarning: pennylane_qiskit.converter: The Measure instruction is not supported by PennyLane, and has not been added to the template.
  warnings.warn(


{'resources': Resources(num_wires=12, num_gates=36, gate_types=defaultdict(<class 'int'>, {'RZ': 2, 'RY': 11, 'CNOT': 23}), gate_sizes=defaultdict(<class 'int'>, {1: 13, 2: 23}), depth=15, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 13, 2: 23}),
 'gate_types': defaultdict(int, {'RZ': 2, 'RY': 11, 'CNOT': 23}),
 'num_operations': 36,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 12,
 'num_trainable_params': 0,
 'depth': 15,
 'num_device_wires': 12,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

In [10]:
print(qml.draw(tapered_circuit_transpiled, expansion_strategy="device")())

 0: ──RZ(3.14)───RY(3.14)────────────────────╭X───────────────────────────────────────────────
 1: ──RZ(3.14)───RY(3.14)─────────────────╭X─│────────────────────────────────────────────────
 2: ──RY(0.53)──╭●──────────────────╭●─╭●─╰●─│─────────────────────╭X─────────────────────────
 3: ──RY(-0.27)─╰X─────────RY(0.27)─│──╰X────│──╭●───────────╭●─╭●─╰●────────────────────╭X───
 4: ────────────────────────────────╰X───────╰●─│────────────│──│──╭X────────────────────│────
 5: ──RY(-0.11)─────────────────────────────────╰X──RY(0.11)─│──╰X─│──╭●───────────╭●─╭●─╰●───
 6: ─────────────────────────────────────────────────────────╰X────╰●─│────────────│──│──╭X───
 7: ──RY(-0.04)───────────────────────────────────────────────────────╰X──RY(0.04)─│──╰X─│──╭●
 8: ───────────────────────────────────────────────────────────────────────────────╰X────╰●─│─
 9: ──RY(-0.29)─────────────────────────────────────────────────────────────────────────────╰X
10: ──────────────────────────────────────────────

## Isotope 20

In [11]:
isotope = 20

ansatz, num_params = ansatz_map[isotope]

@qml.qnode(dev)
@qml.compile(pipeline=pipeline)
def cost(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0))   

params = np.random.uniform(size=num_params)

pprint(qml.specs(cost, expansion_strategy="device")(params))

{'depth': 182,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'gradient',
 'gate_sizes': defaultdict(<class 'int'>, {1: 155, 2: 158}),
 'gate_types': defaultdict(<class 'int'>, {'RZ': 84, 'RY': 71, 'CNOT': 158}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'auto',
 'num_device_wires': 12,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 313,
 'num_trainable_params': 47,
 'num_used_wires': 12,
 'resources': Resources(num_wires=12,
                        num_gates=313,
                        gate_types=defaultdict(<class 'int'>,
                                               {'CNOT': 158,
                                                'RY': 71,
                                                'RZ': 84}),
                        gate_sizes=defaultdict(<class 'int'>, {1: 155, 2: 158}),
                        depth=182,
                        shots=Shots(total_shots=None, shot_vector=()))}


In [12]:
original_tape = cost.qtape
original_qasm = original_tape.to_openqasm()

qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

In [13]:
@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    return qml.expval(qml.PauliZ(0))

In [14]:
qml.specs(tapered_circuit_transpiled)()

{'resources': Resources(num_wires=12, num_gates=312, gate_types=defaultdict(<class 'int'>, {'RZ': 83, 'RY': 71, 'CNOT': 158}), gate_sizes=defaultdict(<class 'int'>, {1: 154, 2: 158}), depth=182, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 154, 2: 158}),
 'gate_types': defaultdict(int, {'RZ': 83, 'RY': 71, 'CNOT': 158}),
 'num_operations': 312,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 12,
 'num_trainable_params': 0,
 'depth': 182,
 'num_device_wires': 12,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Isotope 22

In [15]:
isotope = 22

ansatz, num_params = ansatz_map[isotope]

@qml.qnode(dev)
@qml.compile(pipeline=pipeline)
def cost(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0))   

params = np.random.uniform(size=num_params)

pprint(qml.specs(cost, expansion_strategy="device")(params))

{'depth': 1052,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'gradient',
 'gate_sizes': defaultdict(<class 'int'>, {1: 1146, 2: 787}),
 'gate_types': defaultdict(<class 'int'>, {'RZ': 877, 'RY': 269, 'CNOT': 787}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'auto',
 'num_device_wires': 12,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 1933,
 'num_trainable_params': 190,
 'num_used_wires': 12,
 'resources': Resources(num_wires=12,
                        num_gates=1933,
                        gate_types=defaultdict(<class 'int'>,
                                               {'CNOT': 787,
                                                'RY': 269,
                                                'RZ': 877}),
                        gate_sizes=defaultdict(<class 'int'>,
                                               {1: 1146,
                                                2: 787}),
           

In [16]:
original_tape = cost.qtape
original_qasm = original_tape.to_openqasm()

qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

In [17]:
@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    return qml.expval(qml.PauliZ(0))

In [18]:
qml.specs(tapered_circuit_transpiled)()

{'resources': Resources(num_wires=12, num_gates=1850, gate_types=defaultdict(<class 'int'>, {'RZ': 794, 'RY': 269, 'CNOT': 787}), gate_sizes=defaultdict(<class 'int'>, {1: 1063, 2: 787}), depth=1036, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 1063, 2: 787}),
 'gate_types': defaultdict(int, {'RZ': 794, 'RY': 269, 'CNOT': 787}),
 'num_operations': 1850,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 12,
 'num_trainable_params': 0,
 'depth': 1036,
 'num_device_wires': 12,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Isotope 24

In [19]:
isotope = 24

ansatz, num_params = ansatz_map[isotope]

@qml.qnode(dev)
@qml.compile(pipeline=pipeline)
def cost(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0))   

params = np.random.uniform(size=num_params)

pprint(qml.specs(cost, expansion_strategy="device")(params))

{'depth': 184,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'gradient',
 'gate_sizes': defaultdict(<class 'int'>, {1: 179, 2: 158}),
 'gate_types': defaultdict(<class 'int'>, {'RZ': 96, 'RY': 83, 'CNOT': 158}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'auto',
 'num_device_wires': 12,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 337,
 'num_trainable_params': 47,
 'num_used_wires': 12,
 'resources': Resources(num_wires=12,
                        num_gates=337,
                        gate_types=defaultdict(<class 'int'>,
                                               {'CNOT': 158,
                                                'RY': 83,
                                                'RZ': 96}),
                        gate_sizes=defaultdict(<class 'int'>, {1: 179, 2: 158}),
                        depth=184,
                        shots=Shots(total_shots=None, shot_vector=()))}


In [20]:
original_tape = cost.qtape
original_qasm = original_tape.to_openqasm()

qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

In [21]:
@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    return qml.expval(qml.PauliZ(0))
                
print(np.round(tapered_circuit_transpiled(), decimals=6))

0.775628


In [22]:
qml.specs(tapered_circuit_transpiled)()

{'resources': Resources(num_wires=12, num_gates=334, gate_types=defaultdict(<class 'int'>, {'RZ': 93, 'RY': 83, 'CNOT': 158}), gate_sizes=defaultdict(<class 'int'>, {1: 176, 2: 158}), depth=184, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 176, 2: 158}),
 'gate_types': defaultdict(int, {'RZ': 93, 'RY': 83, 'CNOT': 158}),
 'num_operations': 334,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 12,
 'num_trainable_params': 0,
 'depth': 184,
 'num_device_wires': 12,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}

## Isotope 26

In [23]:
isotope = 26

ansatz, num_params = ansatz_map[isotope]

@qml.qnode(dev)
@qml.compile(pipeline=pipeline)
def cost(params):
    ansatz(params)
    return qml.expval(qml.PauliZ(0))   

params = np.random.uniform(size=num_params)

pprint(qml.specs(cost, expansion_strategy="device")(params))

{'depth': 17,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'gradient',
 'gate_sizes': defaultdict(<class 'int'>, {1: 37, 2: 23}),
 'gate_types': defaultdict(<class 'int'>, {'RZ': 14, 'RY': 23, 'CNOT': 23}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'auto',
 'num_device_wires': 12,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 60,
 'num_trainable_params': 9,
 'num_used_wires': 12,
 'resources': Resources(num_wires=12,
                        num_gates=60,
                        gate_types=defaultdict(<class 'int'>,
                                               {'CNOT': 23,
                                                'RY': 23,
                                                'RZ': 14}),
                        gate_sizes=defaultdict(<class 'int'>, {1: 37, 2: 23}),
                        depth=17,
                        shots=Shots(total_shots=None, shot_vector=()))}


In [24]:
original_tape = cost.qtape
original_qasm = original_tape.to_openqasm()

qiskit_circuit = QuantumCircuit.from_qasm_str(original_qasm)

transpiled_circuit = transpile(
    qiskit_circuit, 
    basis_gates=['rz', 'ry', 'cx', 'measure'],
    optimization_level=3
)                                 

# Convert back to a PennyLane function
new_qfunc = qml.from_qiskit(transpiled_circuit)

In [25]:
@qml.qnode(dev)
def tapered_circuit_transpiled():
    new_qfunc()
    return qml.expval(qml.PauliZ(0))
                
print(np.round(tapered_circuit_transpiled(), decimals=6))

0.982638


In [26]:
print(qml.draw(cost)(np.random.uniform(size=num_params)))

 0: ──RY(-0.02)───────────────────────────────────────────────────────────────────────────────────
 1: ──────────────────────────────────────────────────────────────────────────────────────────────
 2: ──RY(-0.40)──────────────────────────────────────────────────────╭X─────────RY(0.40)─╭X───────
 3: ─────────────────────────────────────────────────────────────────│───────────────────│────────
 4: ──RY(-0.33)────────────────────────╭X─────────RY(0.33)─╭X────────╰●────────╭●────────╰●───────
 5: ───────────────────────────────────│───────────────────│───────────────────╰X────────╭●───────
 6: ──RY(-0.01)─╭X─────────RY(0.01)─╭X─╰●────────╭●────────╰●────────╭●──────────────────│────────
 7: ────────────│───────────────────│────────────╰X────────╭●────────│───────────────────╰X───────
 8: ──RY(0.96)──╰●────────╭●────────╰●─╭●──────────────────│─────────╰X─────────RZ(3.14)──RY(3.14)
 9: ──────────────────────╰X────────╭●─│───────────────────╰X─────────RZ(3.14)──RY(3.14)──────────
10: ──RZ(3

In [27]:
qml.specs(tapered_circuit_transpiled)()

{'resources': Resources(num_wires=12, num_gates=60, gate_types=defaultdict(<class 'int'>, {'RZ': 14, 'RY': 23, 'CNOT': 23}), gate_sizes=defaultdict(<class 'int'>, {1: 37, 2: 23}), depth=17, shots=Shots(total_shots=None, shot_vector=())),
 'gate_sizes': defaultdict(int, {1: 37, 2: 23}),
 'gate_types': defaultdict(int, {'RZ': 14, 'RY': 23, 'CNOT': 23}),
 'num_operations': 60,
 'num_observables': 1,
 'num_diagonalizing_gates': 0,
 'num_used_wires': 12,
 'num_trainable_params': 0,
 'depth': 17,
 'num_device_wires': 12,
 'device_name': 'default.qubit.autograd',
 'expansion_strategy': 'gradient',
 'gradient_options': {},
 'interface': 'auto',
 'diff_method': 'best',
 'gradient_fn': 'backprop'}